In [438]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import mean_squared_error
from scipy import stats
from scipy.stats import t
from sklearn.metrics import confusion_matrix

In [436]:
all_preds = {}
# load in data here
df1 = pd.read_csv("imnemato-youtube[1080p]-1x-vpn-mac-clean-20201031a.csv")
df2 = pd.read_csv("vpn_browsing1.csv")

df3 = pd.read_csv("pgaddiso-youtube-1x-720p-vpn-linux-clean-20201102-1.csv")
df4 = pd.read_csv("vpn_browsing2.csv")


df5 = windows_video_vpn = pd.read_csv("stdoan-youtube[720p60]-1x-vpn-windows-clean-20201102.csv")
df6 = pd.read_csv("vpn_browsing3.csv")

# make sure that dataframes are in list
list_of_frames = [df1, df2, df3,df4,df5,df6]
#add vpn dataframes to next list
video_frames = [df1,df3,df5]
browsing_frames = [df2,df4,df6]

browsing_frames[1]["1->2Bytes"]

0        64
1       132
2        66
3       190
4       167
       ... 
1012      0
1013    190
1014    136
1015    153
1016    136
Name: 1->2Bytes, Length: 1017, dtype: int64

In [412]:
# data cleaning
# helper method to sum all package lengths for each entry in a column
def package(input1):
    sum1 = 0
    for x in input1:
        if x  != "":
            sum1 += int(x)
    return sum1

# adds total pacakge length to all dataframes
def packet_length(frames):
    for j in frames:
        j["sum of sizes"] = pd.Series(j.packet_sizes.apply(lambda x: x.split(";"))).apply(lambda x: package(x))

#label the data
def label():
    for x in video_frames:
            x["Label"] = 1
    for j in browsing_frames:
            j["Label"] = 0


In [413]:
# labes data 
label()

# total packet length
packet_length(list_of_frames)






In [414]:
def ml_model_data(frames):
    # global_varaiables for data storage
    X_train_df =  None
    X_test_df  = None
    y_train_df = None
    y_test_df = None

    for f in frames:
        #isolate features in frames
        X = f[["sum of sizes", "1->2Bytes","2->1Bytes"]]
        
        #isolate label
        y = f[["Label"]]
        
        #split data between training and test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 
       
        #gatther data from all dataframes
        X_train_df = pd.concat([X_train_df, X_train])
        X_test_df = pd.concat([X_test_df, X_test])
        y_train_df = pd.concat([y_train_df, y_train])
        y_test_df = pd.concat([y_test_df, y_test])
        
        
        
    return [X_train_df,X_test_df,y_train_df,y_test_df]
        
    

In [415]:
All_data = ml_model_data(list_of_frames)

In [416]:
def logistic_predictions(ml_model_data):
    clf = LogisticRegression()
    clf.fit(ml_model_data[0], ml_model_data[2])
    predictions = clf.predict(ml_model_data[1]) 
    return predictions 

In [417]:
def svm_predictions(ml_model_data):
    clf = svm.SVC()
    clf.fit(ml_model_data[0], ml_model_data[2])
    predictions = clf.predict(ml_model_data[1])
    return predictions 

In [418]:
def bayes_predictions(ml_model_data):
    clf =  GaussianNB()
    clf.fit(ml_model_data[0], ml_model_data[2])
    predictions = clf.predict(ml_model_data[1])
    return predictions 

In [419]:
def k_predictions(ml_model_data):
    clf = KNeighborsClassifier(n_neighbors=3)
    clf.fit(ml_model_data[0], ml_model_data[2])
    predictions = clf.predict(ml_model_data[1])
    return predictions 

In [420]:
k_preds = k_predictions(All_data)
all_preds["k"] = k_preds

C:\Users\dania\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [421]:
bayes_preds = bayes_predictions(All_data)
all_preds["bayes"] = bayes_preds

C:\Users\dania\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [422]:
svm_preds = svm_predictions(All_data)
all_preds["svm"] = svm_preds

C:\Users\dania\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dania\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [423]:
predictions = logistic_predictions(All_data)
all_preds["LogisticRegression"] = predictions

C:\Users\dania\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\dania\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [424]:
def confidence_interval(prediction, confidence):
    min_value = min(prediction)
    max_value = max(prediction)
    
    average_list = []
    
    for x in range(1000):
        samples = np.random.choice(prediction, len(prediction))
        average_list.append(np.mean(samples))
    
    size = len(average_list)
    average = np.mean(average_list)
    st_error = stats.sem(average_list)
    interval = st_error * t.ppf((1 + confidence)/2,size-1)
    low = round(average- interval)
    up = round(average +interval)
    return (low, up)

In [425]:
for x in all_preds:
    percent_correct = sum(np.array(All_data[3]["Label"]) == np.array(all_preds[x]))/len(All_data[3]["Label"])
    print(str(x) +" "+ str(percent_correct) + " correct")

k 0.8604155567394779 correct
bayes 0.7384123601491742 correct
svm 0.8614810868407032 correct
LogisticRegression 0.2749067661161428 correct


In [426]:
for x in all_preds:
    print("The mean squared error of "+str(x) + " "+ str(mean_squared_error(All_data[3]["Label"], all_preds[x])))

The mean squared error of k 0.1395844432605221
The mean squared error of bayes 0.2615876398508258
The mean squared error of svm 0.13851891315929676
The mean squared error of LogisticRegression 0.7250932338838573


## The best Model is K means, now time to see what features we can change in model

In [427]:
def ml_model_data1(frames):
    # global_varaiables for data storage
    X_train_df =  None
    X_test_df  = None
    y_train_df = None
    y_test_df = None

    for f in frames:
        #isolate features in frames
        X = f[["sum of sizes", "1->2Bytes", "Port1"]]
        
        #isolate label
        y = f[["Label"]]
        
        #split data between training and test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 
       
        #gatther data from all dataframes
        X_train_df = pd.concat([X_train_df, X_train])
        X_test_df = pd.concat([X_test_df, X_test])
        y_train_df = pd.concat([y_train_df, y_train])
        y_test_df = pd.concat([y_test_df, y_test])
        
        
        
    return [X_train_df,X_test_df,y_train_df,y_test_df]
        
    

In [428]:
def k_predictions1(ml_model_data):
    clf = KNeighborsClassifier(n_neighbors=3)
    clf.fit(ml_model_data[0], ml_model_data[2])
    predictions = clf.predict(ml_model_data[1])
    return predictions 

In [429]:
All_data = ml_model_data1(list_of_frames)
new_k = k_predictions1(All_data)
percent_correct = sum(np.array(All_data[3]["Label"]) == np.array(new_k))/len(All_data[3]["Label"])
percent_correct

C:\Users\dania\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.9057005860415557

In [442]:
tn, fp, fn, tp = confusion_matrix(All_data[3]["Label"], new_k).ravel()

In [447]:
print("The true negatives " + str(tn))
print("The true positives " + str(tp))
print("The False positives " + str(fp))
print("The False negatives " + str(fn))

The true negatives 1285
The true positives 415
The False positives 76
The False negatives 101


In [479]:
All_data[3][All_data[3]["Label"] != new_k].head()

,Label
250,1
153,1
72,1
252,1
362,1


In [480]:
new_k[:10]

array([1, 1, 0, 1, 0, 1, 1, 1, 1, 0], dtype=int64)

In [481]:
np.array(All_data[3]["Label"][:10])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

I seem to have false positives at index 2 and 4, and 9.  Looking at training data  I get these False postives when there is sudden change in data. The data makes labels based on the data points near it. Since there is a change in the data it ruins the pattern having it classified wrong. Kmeans looks at the algorithim near every cluster hence data fluxations can mess up algorithm when training the data.